In [ ]:
from google.colab import auth
auth.authenticate_user()

PROJECT_ID = "genai-poc-424806"   # your project
LOCATION = "us"
print("OK:", PROJECT_ID, LOCATION)


In [ ]:
import requests
import json
from google.colab import auth
auth.authenticate_user()

# Get access token
access_token = !gcloud auth print-access-token
access_token = access_token[0]

# Create connection via REST API
url = f"https://bigqueryconnection.googleapis.com/v1/projects/genai-poc-424806/locations/US/connections"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
data = {
    "connectionId": "vertex_conn",
    "cloudResource": {}
}

response = requests.post(url, headers=headers, json=data)
print(f"Response: {response.status_code}")
print(response.json())

In [ ]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
!gcloud config set project genai-poc-424806
!gcloud auth list


In [ ]:
!bq --project_id=genai-poc-424806 mk --connection --location=US \
  --connection_type=CLOUD_RESOURCE vertex_conn


In [ ]:
!bq --project_id=genai-poc-424806 ls --connection --location=US

In [ ]:
!bq show --format=prettyjson genai-poc-424806:google_patents_research | grep location


In [ ]:
!bq ls --project_id=genai-poc-424806


In [ ]:
!bq ls genai-poc-424806:patent_demo


In [ ]:
%%bigquery --project genai-poc-424806
CREATE OR REPLACE MODEL `patent_demo.embedding_model`
  REMOTE WITH CONNECTION `us.vertex_conn`
  OPTIONS (ENDPOINT = 'text-embedding-004');  -- or 'gemini-embedding-001'


In [ ]:
%%bigquery --project genai-poc-424806
CREATE TABLE IF NOT EXISTS `patent_demo.patent_embeddings` (
  publication_number STRING,
  title STRING,
  abstract STRING,
  text_embedding ARRAY<FLOAT64>
);


In [ ]:
%%bigquery --project genai-poc-424806
SELECT column_name, data_type
FROM `genai-poc-424806.patent_demo`.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'patent_docs'
ORDER BY ordinal_position;


In [ ]:
%%bigquery --project genai-poc-424806
CREATE OR REPLACE TABLE `patent_demo.patent_embeddings` AS
WITH src AS (
  SELECT
    d.publication_number,
    CAST(d.title    AS STRING) AS title_text,
    CAST(d.abstract AS STRING) AS abstract_text
  FROM `patent_demo.patent_docs` d
),
gen AS (
  SELECT
    publication_number,
    ml_generate_embedding_result AS text_embedding
  FROM ML.GENERATE_EMBEDDING(
    MODEL `patent_demo.embedding_model`,
    (
      SELECT
        publication_number,
        CONCAT(COALESCE(title_text, ''), ' ', COALESCE(abstract_text, '')) AS content
      FROM src
      WHERE (title_text IS NOT NULL OR abstract_text IS NOT NULL)
      ORDER BY publication_number
      -- LIMIT 1000  -- ← uncomment for a pilot run first
    )
  )
)
SELECT
  s.publication_number,
  s.title_text   AS title,
  s.abstract_text AS abstract,
  g.text_embedding
FROM gen g
JOIN src s USING (publication_number);


In [ ]:
%%bigquery --project genai-poc-424806
SELECT COUNT(*) AS rows_with_embeddings FROM `patent_demo.patent_embeddings`;


In [ ]:
import os
PROJECT_ID = "genai-poc-424806"   # replace with your actual project
os.environ["PROJECT_ID"] = PROJECT_ID


In [ ]:
%%bigquery --project genai-poc-424806
CREATE VECTOR INDEX `patent_demo.idx_patent_embeddings`
ON `patent_demo.patent_embeddings`(text_embedding)
STORING(publication_number, title, abstract)               -- optional but recommended
OPTIONS(
  index_type = 'IVF',
  distance_type = 'COSINE',
  ivf_options = '{"num_lists": 500}'                       -- tune as needed (≤ 5000)
);


In [ ]:
%%bigquery --project genai-poc-424806
-- High-level: one row per vector index in the dataset
SELECT *
FROM `patent_demo`.INFORMATION_SCHEMA.VECTOR_INDEXES;


In [ ]:
%%bigquery --project genai-poc-424806
-- Options actually applied (index_type, distance_type, IVF params, etc.)
SELECT *
FROM `patent_demo`.INFORMATION_SCHEMA.VECTOR_INDEX_OPTIONS;

In [ ]:
%%bigquery --project genai-poc-424806
DECLARE user_query STRING DEFAULT 'battery thermal runaway prevention in EVs';

-- Build a "query table" with the same column name as the base embedding column
WITH q AS (
  SELECT ml_generate_embedding_result AS text_embedding
  FROM ML.GENERATE_EMBEDDING(
    MODEL `patent_demo.embedding_model`,
    (SELECT user_query AS content)
  )
)
SELECT
  base.publication_number,
  base.title,
  base.abstract,
  distance
FROM VECTOR_SEARCH(
  TABLE `patent_demo.patent_embeddings`,   -- base table
  'text_embedding',                         -- embedding column in base
  TABLE q,                                  -- one-row query table with matching column name
  top_k => 8,
  distance_type => 'COSINE'
);


In [ ]:
%%bigquery --project genai-poc-424806
CREATE OR REPLACE MODEL `patent_demo.gemini_text`
REMOTE WITH CONNECTION `us.vertex_conn`
OPTIONS (ENDPOINT = 'gemini-2.0-flash');


In [ ]:
%%bigquery --project genai-poc-424806
DECLARE user_query STRING DEFAULT 'Summarize top innovations that reduce EV battery fire risk.';

WITH q AS (
  SELECT ml_generate_embedding_result AS text_embedding
  FROM ML.GENERATE_EMBEDDING(
    MODEL `patent_demo.embedding_model`,
    (SELECT user_query AS content)
  )
),
hits AS (
  SELECT
    base.publication_number,
    base.title,
    SUBSTR(base.abstract, 1, 1200) AS abstract,
    distance
  FROM VECTOR_SEARCH(
    TABLE `patent_demo.patent_embeddings`,
    'text_embedding',
    TABLE q,
    top_k => 8,
    distance_type => 'COSINE'
  )
  ORDER BY distance
),
context AS (
  SELECT STRING_AGG(
    CONCAT('PUB: ', publication_number, '\nTITLE: ', title, '\nABSTRACT: ', abstract),
    '\n\n---\n\n'
    ORDER BY distance
  ) AS ctx
  FROM hits
)
SELECT *
FROM ML.GENERATE_TEXT(
  MODEL `patent_demo.gemini_text`,
  (SELECT CONCAT(
      'You are a precise patent analyst. Use ONLY the CONTEXT. ',
      'If info is missing, say so. Provide bullet points and cite PUB IDs used.\n\n',
      'QUESTION: ', user_query, '\n\n',
      'CONTEXT:\n', ctx
  ) AS prompt FROM context),
  STRUCT(0.2 AS temperature, 900 AS max_output_tokens)
);


In [ ]:
!bq --project_id=genai-poc-424806 ls -m patent_demo


In [ ]:
# === Minimal RAG UI for BigQuery + Vertex AI (Colab) ===
# Type a query -> get answer (optionally show sources)

import os
from google.cloud import bigquery
import ipywidgets as widgets
from IPython.display import display, Markdown
import pandas as pd

# ------- Configure your project / dataset / models / table -------
PROJECT_ID   = "genai-poc-424806"
DATASET      = "patent_demo"
EMB_MODEL    = f"{DATASET}.embedding_model"   # text-embedding-004
LLM_MODEL    = f"{DATASET}.gemini_text"       # gemini-2.0-flash
EMB_TABLE    = f"{DATASET}.patent_embeddings" # has column text_embedding ARRAY<FLOAT64>

# UI defaults (you can tweak live)
DEFAULT_QUERY      = "battery thermal runaway prevention in EVs"
TOP_K_DEFAULT      = 5          # 3–8 is a good range
MAX_CTX_CHARS      = 1200       # keep abstracts compact to avoid token limits
TEMPERATURE        = 0.2        # LITERAL gets embedded into SQL (TVF requires literals)
MAX_OUTPUT_TOKENS  = 900        # LITERAL
SHOW_SOURCES       = True       # set False to hide the sources table

# -----------------------------------------------------------------
os.environ["GCLOUD_PROJECT"] = PROJECT_ID
bq = bigquery.Client(project=PROJECT_ID)

def run_rag(user_query: str,
            top_k: int = TOP_K_DEFAULT,
            max_ctx_chars: int = MAX_CTX_CHARS,
            temperature: float = TEMPERATURE,
            max_output_tokens: int = MAX_OUTPUT_TOKENS):
    """
    Runs retrieval -> generation inside BigQuery.
    Notes:
      - user_query is passed as a parameter
      - TVF settings (temperature, max_output_tokens) MUST be literal constants, so we embed them.
    """
    # --- Main query: retrieve -> build context -> generate ---
    sql_answer = f"""
    DECLARE user_query STRING DEFAULT @user_query;

    WITH q AS (
      SELECT ml_generate_embedding_result AS text_embedding
      FROM ML.GENERATE_EMBEDDING(
        MODEL `{EMB_MODEL}`,
        (SELECT user_query AS content)
      )
    ),
    hits AS (
      SELECT
        base.publication_number,
        base.title,
        SUBSTR(base.abstract, 1, {max_ctx_chars}) AS abstract,
        distance
      FROM VECTOR_SEARCH(
        TABLE `{EMB_TABLE}`,
        'text_embedding',
        TABLE q,
        top_k => {top_k},
        distance_type => 'COSINE'
      )
      ORDER BY distance
    ),
    context AS (
      SELECT STRING_AGG(
        CONCAT('PUB: ', publication_number, '\\nTITLE: ', title, '\\nABSTRACT: ', abstract),
        '\\n\\n---\\n\\n' ORDER BY distance
      ) AS ctx
      FROM hits
    ),
    gen AS (
      SELECT
        ml_generate_text_result AS gen_json
      FROM ML.GENERATE_TEXT(
        MODEL `{LLM_MODEL}`,
        (
          SELECT CONCAT(
            'You are a precise patent analyst. Use ONLY the CONTEXT. ',
            'If info is missing, say so. Provide bullet points and cite PUB IDs.\\n\\n',
            'QUESTION: ', user_query, '\\n\\n',
            'CONTEXT:\\n', ctx
          ) AS prompt
          FROM context
        ),
        STRUCT({temperature} AS temperature, {max_output_tokens} AS max_output_tokens)  -- must be literals
      )
    )
    SELECT JSON_VALUE(gen_json, '$.candidates[0].content.parts[0].text') AS answer
    FROM gen;
    """

    job = bq.query(
        sql_answer,
        job_config=bigquery.QueryJobConfig(
            query_parameters=[bigquery.ScalarQueryParameter("user_query", "STRING", user_query)]
        ),
        location="US",
    )
    answer_df = job.result().to_dataframe()

    # --- Optional: fetch sources for display ---
    sources_df = pd.DataFrame()
    if SHOW_SOURCES:
        sql_sources = f"""
        DECLARE user_query STRING DEFAULT @user_query;

        WITH q AS (
          SELECT ml_generate_embedding_result AS text_embedding
          FROM ML.GENERATE_EMBEDDING(
            MODEL `{EMB_MODEL}`,
            (SELECT user_query AS content)
          )
        )
        SELECT
          base.publication_number,
          base.title,
          distance
        FROM VECTOR_SEARCH(
          TABLE `{EMB_TABLE}`,
          'text_embedding',
          TABLE q,
          top_k => {top_k},
          distance_type => 'COSINE'
        )
        ORDER BY distance;
        """
        sources_job = bq.query(
            sql_sources,
            job_config=bigquery.QueryJobConfig(
                query_parameters=[bigquery.ScalarQueryParameter("user_query", "STRING", user_query)]
            ),
            location="US",
        )
        sources_df = sources_job.result().to_dataframe()

    answer_text = (answer_df.iloc[0]["answer"] if not answer_df.empty else "")
    return answer_text, sources_df

# ----------------------------- UI -----------------------------
qbox = widgets.Textarea(
    value=DEFAULT_QUERY,
    placeholder="Type your question…",
    description="Query:",
    layout=widgets.Layout(width="100%", height="90px")
)
topk_slider = widgets.IntSlider(value=TOP_K_DEFAULT, min=1, max=10, step=1, description='top_k:')
run_btn = widgets.Button(description="Run", button_style="primary")
out = widgets.Output()

def on_click(_):
    out.clear_output()
    with out:
        print("Running…")
        try:
            answer, sources = run_rag(
                user_query=qbox.value,
                top_k=topk_slider.value,
                max_ctx_chars=MAX_CTX_CHARS,
                temperature=TEMPERATURE,
                max_output_tokens=MAX_OUTPUT_TOKENS
            )
            display(Markdown("### Answer"))
            display(Markdown(answer if isinstance(answer, str) and answer else "_(no text)_"))
            if SHOW_SOURCES:
                display(Markdown("### Sources (top-k)"))
                display(sources)
        except Exception as e:
            print("ERROR:", e)

run_btn.on_click(on_click)
display(qbox, topk_slider, run_btn, out)
